In [1]:
import numpy as np
from astropy.table import Table, vstack, Column
from astropy.io import fits, ascii
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pdb
import warnings

In [2]:
path = '.\\..\\catalogs\\'
fileDE = Table.read(path+'VDGC_pPXF_2017_v2.fits')
fileUDG = Table.read(path+'VUGC_pPXF_2017_v2.fits')

In [3]:
warnings.simplefilter("ignore", RuntimeWarning)
def spec_interp(wv,fx,nwwv,*args):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned
    #   - nwwv: new wavelength array
    #
    #Optional argument: variance
    npix = len(wv)
    if len(args) == 0:
        var = np.ones(npix)
        nwvarFlag = False
    else:
        var = args[0]
        nwvarFlag = True
    nwpix = len(nwwv)
    #Calculate wavelength endpoints for each pixel
    wvl = (wv + np.roll(wv,1))/2.
    wvh = (wv + np.roll(wv,-1))/2.
    wvl[0] = wv[0] - (wv[1] - wv[0])/2.
    wvh[npix-1] = wv[npix-1] + (wv[npix-1]-wv[npix-2])/2.
    #Calculate endpoints of the final array
    bwv = np.zeros(nwpix+1)
    bwv[0:nwpix] = (nwwv+np.roll(nwwv,1))/2.
    bwv[0] = nwwv[0] - (nwwv[1] - nwwv[0])/2.
    bwv[nwpix] = nwwv[nwpix-1]+(nwwv[nwpix-1] - nwwv[nwpix - 1])/2.
    #Create tmp arrays for final array
    nwfx = np.zeros(nwpix)
    nwvar = np.zeros(nwpix)
    nwunitfx = np.zeros(nwpix)
    #Loop through the arrays
    for q in range(npix):
        #No overlap
        if (wvh[q] <= bwv[0]) | (wvl[q] >= bwv[nwpix]):
            continue
        #Find pixel that bw is within
        if wvl[q] <= bwv[0]:
            i1 = [0]
        else:
            i1 = np.argwhere((wvl[q] <= np.roll(bwv,-1)) & (wvl[q] > bwv))[0]
        if wvh[q] > bwv[nwpix]:
            i2 = [nwpix-1]
        else:
            i2 = np.argwhere((wvh[q] <= np.roll(bwv,-1)) & (wvh[q] > bwv))[0]
        j1 = i1[0]
        j2 = i2[0]
        #Now Sum up
        for kk in range(j1,j2+1):
            #Rejected pixesl do not get added in
            if var[q] > 0.:
                frac = ( np.min([wvh[q],bwv[kk+1]]) - np.max([wvl[q],bwv[kk]]) ) / (wvh[q]-wvl[q])
                nwfx[kk] = nwfx[kk]+frac*fx[q]
                nwunitfx[kk] = nwunitfx[kk]+frac*1.0
                #Variance
                if nwvarFlag:
                    if (var[q] <= 0.) | (nwvar[kk] == -1):
                       nwvar[kk] = -1
                    else:
                       nwvar[kk] = nwvar[kk]+frac*var[q]
    if nwvarFlag:
        fxOut = nwfx/nwunitfx
        varOut = nwvar*nwunitfx
        
        return fxOut,varOut
    else:
        fxOut = nwfx/nwunitfx
        return fxOut


def rebinspec(*args,**kwargs):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned
    #   - nwwv: new wavelength array
    #
    #Optional arguments:
    #   - var = var, input and output variance
    #   - ivar = ivar, input and output ivar

    if len(args) != 3:
        print('Proper syntax is: out = rebinspec(wv, fx, nwwv, **kwargs)')
        return np.nan

    else:
        wv, fx, nwwv = args

        var = kwargs.get('var',None)
        ivar = kwargs.get('ivar',None)

        if (var is not None) & (ivar is None):
            nwfx,nwvar = spec_interp(wv,fx,nwwv,var)

            return nwfx, nwvar
        elif (var is None) & (ivar is not None):
            var = 1./ivar
            nwfx,nwvar_1 = spec_interp(wv,fx,nwwv,var)
            nwvar_1[nwvar_1 == 0.0] = -10.0
            nwivar = 1.0/nwvar_1
            nwivar[nwivar < 0.0] = 0.0
            
            return nwfx, nwivar
        else:
            nwfx = spec_interp(wv,fx,nwwv)

            return nwfx

In [4]:
columns_to_change = ['ZERR', 'ZERR_ne', 'ZERR_pe', 'ZHEL', 'ZOBS', 'ZOBS_ne', 'ZOBS_pe']
columns_to_change2 = ['HELCOR','ABANDCOR', 'ABANDCOR_ne', 'ABANDCOR_pe']     #rename columns for the DE
for col in columns_to_change:
    fileDE[col] = fileDE[col] * 3e+5
    fileDE.rename_column(col, 'V' + col[1:]) 
for col in columns_to_change2:
    fileDE[col] = fileDE[col] * 3e+5

In [5]:
DE_GC = fileDE[fileDE['ZCONF']==1]      #cuts down to ZCONF = 1
DE_GC = DE_GC[DE_GC['GCSAT']== 'Y']     #cuts down to GCSAT = yes
UDG_GC = fileUDG[fileUDG['ZCONF']==1]
UDG_GC = UDG_GC[UDG_GC['GCSAT']== 'Y']
intermediate = UDG_GC.copy()
udg_bool = (UDG_GC['HOST'] == 'VLSB-B') | (UDG_GC['HOST'] == 'VLSB-D') | (UDG_GC['HOST'] == 'VCC0615')
UDG_GC= UDG_GC[udg_bool]
belongDGC = ~udg_bool
DE_GC = vstack([DE_GC, intermediate[belongDGC]])     #combines DE_GC with the rows originally from UDG data
# DE_GC.remove_rows(np.nonzero((DE_GC['VHEL'] > -300) & (DE_GC['VHEL'] < 300))[0])     #removes potential milky way stars
# UDG_GC.remove_rows(np.no vgvvggggggggggggggggggggggggggggggggginzero((UDG_GC['VHEL'] > -300) & (UDG_GC['VHEL'] < 300))[0])

In [6]:
fileOrphanDE = Table.read(path+'orphanGCs.fits')
filekinDE = Table.read(path+'VDGC_kinematic_prob.fits')
fileOrphanUDG = Table.read(path+'SelectGCsTrue2.fits')
filekinUDG = Table.read(path+'SelectGCsTrue_kinematic_prob.fits')
def typeandprob(file):
    probability = []
    objecttype = []
    for row in file:
        probability.append(max(row))
        vals = []
        for val in row:
            vals.append(val)
        if (vals.index(max(vals))== 0):
            objecttype.append('Star')
        elif (vals.index(max(vals))== 1):
            objecttype.append('ICGC')
        elif (vals.index(max(vals))== 2):
            objecttype.append('M87GC')
    return probability, objecttype
probabilityDE, objecttypeDE = typeandprob(filekinDE)
probabilityUDG, objecttypeUDG = typeandprob(filekinUDG)
fileOrphanDE['Probability'] = probabilityDE
fileOrphanUDG['Probability'] = probabilityUDG
fileOrphanDE['Type'] = objecttypeDE
fileOrphanUDG['Type'] = objecttypeUDG

In [7]:
allnoDE = fileDE[(fileDE['ZCONF']==1) & ((fileDE['GCSAT'] == 'N') | (fileDE['GCSAT'] == 'M'))]        #gets all GCSATs that are NO and ZCONF that are 1 for DE
allnoUDG = fileUDG[(fileUDG['ZCONF']==1) & ((fileUDG['GCSAT'] == 'N') | (fileUDG['GCSAT'] == 'M'))]     #does same for the UDG
allnoDE.sort('SPEC1DNAME')     #organizes based on SPEC1DNAME for all tables, now since they are all sorted by the same manner their rows must correspond to one another
allnoUDG.sort('SPEC1DNAME')
fileOrphanDE.sort('SPEC1DNAME')
fileOrphanUDG.sort('SPEC1DNAME')
focusedUDG = fileOrphanUDG['SPEC1DNAME','Probability','Type']
focusedUDG.add_row(['spec1d.vugc1.044.GCXO0504.fits.gz',-1,'Unkwn'])
focusedUDG.sort('SPEC1DNAME')
allnoDE['Probability'] = probabilityDE
allnoUDG['Probability'] = focusedUDG['Probability']
allnoDE['Type'] = objecttypeDE
allnoUDG['Type'] = focusedUDG['Type']

In [8]:
DEtype = []
DEprob = []
UDGtype = []
UDGprob = []
for i in range(len(DE_GC)):
    DEtype.append('DE_GC')
    DEprob.append(1)
for i in range(len(UDG_GC)):
    UDGtype.append('UDG_GC')
    UDGprob.append(1)
DE_GC['Type'] = DEtype
DE_GC['Probability'] = DEprob
UDG_GC['Type'] = UDGtype
UDG_GC['Probability'] = UDGprob

In [9]:
finaltable = Table()
finaltable = vstack([DE_GC, UDG_GC])  
finaltable = vstack([finaltable, allnoDE])     
finaltable = vstack([finaltable, allnoUDG])
allnames = []
test = Table()
finaltable.sort('SPEC1DNAME')
for i in range (len(finaltable)):
    allnames.append(finaltable[i]['SPEC1DNAME'])
    allnames[i] = allnames[i][:-3]

In [10]:
nucDat = ascii.read(path + 'nuc.dat')
nucDat.sort('VCC')
finaltable.sort('VCC')
myVCC = []
nucVCC = []
for i in range (len(finaltable)):
    if (finaltable[i]['VCC'] == '---'):
        pass
    else:
        myVCC.append(finaltable[i]['VCC'])
for i in range(len(nucDat)):
    nucVCC.append(''+(str(nucDat[i]['name'][24:])))
for i in range(len(nucVCC)):
    if (nucVCC[i][0] == 'V'):
        pass
    else:
        nucVCC[i] = 'V' + nucVCC[i]
indexmatches = []
valmatches = []
sub = 'VCC'
UDGtingz = []
UDGtingz.append([s for s in myVCC if sub not in s])
UDGtingz = UDGtingz[0]
for i in range(len(UDGtingz)):
    myVCC.remove(UDGtingz[i])
zerocolon = []
for val in myVCC:
    if val not in nucVCC:
        zerocolon.append(val)
for val in zerocolon:
    myVCC.remove(val)
for val in myVCC:
    if val in nucVCC:
        valmatches.append(val)

In [11]:
nuclei = list(nucDat['Nucleus'])

In [12]:
finaltable['Nucleus'] = ' 0'
i = 0
for row in finaltable:
    if row['VCC'] == myVCC[i]:
        row['Nucleus'] = nuclei[i]
        row['Type'] = 'DE_NUC'
        if (i < len(myVCC)- 1):
            i += 1
        else:
            pass
    elif row['VCC'] in zerocolon:
        row['Nucleus'] = ':'
    elif row['VCC'] in UDGtingz:
        if row['TARGTYPE'] == 'NUC':
            row['Nucleus'] = '1'
        else:
            row['Nucleus'] = '0' 
    else:
        pass
for name in finaltable:
    name['SPEC1DNAME'] = name['SPEC1DNAME'][:-3]
finaltable

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Type,Probability,Nucleus
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,bytes20,int16,int16,int16,int16,int16,bytes40,str6,float64,str2
---,ALG,N,---,187.0097,10.359333,54.56785,2.5169947,1.2761084,1.2873586,1.0,117.61137,0.21014537,0.21014537,48.443516,0.14009692,0.2201523,14.6,196.03,alg42,vdgc1,0,0,-1,15,21,spec1d.vdgc1.000.alg42.fits,Star,0.9815727472305298,0
---,GCN,N,---,190.27808,11.347694,188.05902,3.7075615,3.0438979,2.974683,1.0,176.13185,2.5017307,2.481717,-24.827175,1.2008307,1.0607338,12.9,16.66,gcN188,vdgc8,42,51,4,4,10,spec1d.vdgc8.042.gcN188.fits,Star,0.9456380605697632,0
---,GCN,N,---,190.29584,11.295389,61.981804,2.5828428,1.3721166,1.4436005,1.0,46.822395,0.5003461,0.65045,-28.059412,0.26018,0.31021464,12.9,60.51,gcN168,vdgc8,40,48,4,200,205,spec1d.vdgc8.040.gcN168.fits,Star,0.9810741543769836,0
---,GCN,N,---,190.26013,11.257055,1412.1558,7.218932,6.1288867,7.670063,1.0,1400.5588,5.1235447,5.9441123,-24.496948,3.12216,4.68324,12.9,6.76,gcN150,vdgc8,38,44,4,72,78,spec1d.vdgc8.038.gcN150.fits,ICGC,1.0,0
---,GCN,N,---,190.31033,11.256194,553.4618,4.6429586,4.5116687,3.7755418,1.0,539.77344,4.2129145,3.1021461,-26.588394,1.0207062,1.7512115,12.9,15.38,gcN149,vdgc8,37,41,4,125,131,spec1d.vdgc8.037.gcN149.fits,ICGC,0.9991095662117004,0
---,GCN,N,---,190.29695,11.242444,217.64949,7.3605747,6.507694,7.5643997,1.0,212.53703,6.0341744,7.1249294,-18.01246,2.0914469,2.21153,12.9,5.84,gcN141,vdgc8,36,40,4,19,25,spec1d.vdgc8.036.gcN141.fits,Star,0.9189153909683228,0
---,GCN,N,---,190.27217,11.235723,687.12427,6.7171874,4.529523,8.342857,1.0,673.9462,3.1621876,7.4251366,-26.078041,2.99207,3.5924854,12.9,5.36,gcN136,vdgc8,35,39,1,21,27,spec1d.vdgc8.035.gcN136.fits,ICGC,0.9999980330467224,0
---,GCN,N,---,190.27824,11.225139,4.3819556,2.957761,1.9460571,2.0825222,1.0,-6.1842785,1.3509345,1.5010384,-23.466234,0.6304362,0.72049844,12.9,29.78,gcN130,vdgc8,34,38,4,29,35,spec1d.vdgc8.034.gcN130.fits,Star,0.9828998446464539,0
---,GCN,N,---,190.27042,11.220639,-113.08931,4.0795856,3.6248136,3.306391,1.0,-119.50267,2.952042,2.851973,-19.31336,1.69117,1.1107684,12.9,15.27,gcN126,vdgc8,33,37,4,25,30,spec1d.vdgc8.033.gcN126.fits,Star,0.9704931974411011,0


In [13]:
path = '.\\..\\spectra\\'
vdgcfiles = []
vugcfiles = []
for filename in os.listdir(path):
    if ((filename in allnames) | (filename[:-3] in allnames)):
        if ((filename.endswith(".fits")) & ("spec" in filename)): 
            vdgcfiles.append(filename)
        elif ((filename.endswith(".gz")) & ("spec" in filename)): 
            vugcfiles.append(filename[:-3])

In [14]:
def infof(file):
    file.info()
    file[1].data['SPEC']

In [15]:
# correct = np.arange(4000,8750.9,0.9)
# newflux = []
# newvar = []
# for i in range(len(testlambdas)):  
#     newflux, newvar = rebinspec(testlambdas[i][0],testfluxes[i][0], correct, ivar = testvariances[i][0])

In [16]:
vobsidxDE = []
vobsDE = []
finaltable.sort('SPEC1DNAME')
for file in vdgcfiles:
    print(file)
    for ind, val in enumerate(allnames):
        if file == val:
            print(val)
            print(ind)
            vobsidxDE.append(ind)
for index in vobsidxDE:
    vobsDE.append(finaltable[index]['VOBS'])
vobsidxUDG = []
vobsUDG = []
for file in vugcfiles:
    for ind, val in enumerate(allnames):
        if file == val:
            vobsidxUDG.append(ind)
for index in vobsidxUDG:
    vobsUDG.append(finaltable[index]['VOBS'])

spec1d.vdgc1.000.alg42.fits
spec1d.vdgc1.000.alg42.fits
0
spec1d.vdgc1.001.alg115.fits
spec1d.vdgc1.001.alg115.fits
1
spec1d.vdgc1.002.alg140.fits
spec1d.vdgc1.002.alg140.fits
2
spec1d.vdgc1.003.alg164.fits
spec1d.vdgc1.003.alg164.fits
3
spec1d.vdgc1.004.alg177.fits
spec1d.vdgc1.004.alg177.fits
4
spec1d.vdgc1.005.alg191.fits
spec1d.vdgc1.005.alg191.fits
5
spec1d.vdgc1.006.alg194.fits
spec1d.vdgc1.006.alg194.fits
6
spec1d.vdgc1.007.nuc5.fits
spec1d.vdgc1.007.nuc5.fits
7
spec1d.vdgc1.008.nuc6.fits
spec1d.vdgc1.008.nuc6.fits
8
spec1d.vdgc1.009.nuc7.fits
spec1d.vdgc1.009.nuc7.fits
9
spec1d.vdgc1.011.gcA10.fits
spec1d.vdgc1.011.gcA10.fits
10
spec1d.vdgc1.012.gcA17.fits
spec1d.vdgc1.012.gcA17.fits
11
spec1d.vdgc1.014.gcA19.fits
spec1d.vdgc1.014.gcA19.fits
12
spec1d.vdgc1.015.gcA20.fits
spec1d.vdgc1.015.gcA20.fits
13
spec1d.vdgc1.020.gcN82.fits
spec1d.vdgc1.020.gcN82.fits
14
spec1d.vdgc1.022.gcN87.fits
spec1d.vdgc1.022.gcN87.fits
15
spec1d.vdgc1.023.gcN95.fits
spec1d.vdgc1.023.gcN95.fits
16
s

In [21]:
alllambdas = []
allfluxes = []
allvariances = []
def coaddition(redorblue):
    for i in range(len(vdgcfiles)):
        f = fits.open(path + vdgcfiles[i])
        try:
            temp = f['Bxspf-' + redorblue]
        except:
            temp = f['Horne-' + redorblue]
        newwave = temp.data['LAMBDA']
        oldwave = newwave/(1 + ((vobsDE[i]/300000)))
        alllambdas.append(oldwave)
        allfluxes.append(temp.data['SPEC'])
        allvariances.append(temp.data['IVAR'])
        testlambdas = alllambdas[:10]
        testfluxes = allfluxes[:10]
        testvariances = allvariances[:10]
        return alllambdas, allfluxes, allvariances, testlambdas, testfluxes, testvariances
alllambdasB, allfluxesB, allvariancesB, testlambdasB, testfluxesB, testvariancesB = coaddition('B')
alllambdasR, allfluxesR, allvariancesR, testlambdasR, testfluxesR, testvariancesR = coaddition('R')

In [23]:
correct = np.arange(4000,8750.9,0.9)
newflux = []
newvar = [] 
for i in range(len(testlambdas)):  
    newflux, newvar = np.nan_to_num(rebinspec(testlambdas[i][0],testfluxes[i][0], correct, ivar = testvariances[i][0]))